In [172]:
import numpy as np
import scipy.io
import torch
import torch.nn.functional as F

from torch.utils.data import DataLoader
import os
import pandas as pd
import matplotlib.pyplot as plt

data from https://www.kaggle.com/datasets/amananandrai/complete-eeg-dataset/data

durée = 60s


freq = 516 Hz

In [142]:
labels=np.array([0,1,1,1,0,1,0,1,1,0,0,1,1,1,0,1,1,1,1,0,1,0,0,1,1,1,1,1,1,1,0,1,1,1,1,1])
len(labels)

36

In [143]:
counts = [str(i) for i in range(31)]
counts = ['0' + i if len(i) == 1 else i for i in counts]


In [144]:
# set construct_files to True if you want to construct the files
construct_files = False
if construct_files : 
    np.random.seed(0)

    for file_name in counts : 
        df = pd.read_csv('./../data/kaggle_2/s'+file_name+'.csv', header=None).transpose().to_numpy()
        for i in range(31*4): 
            sample = pd.DataFrame(df[:, 250*i:250*i+1000]) # slicing of 250 
            random_float = np.random.rand()
            if random_float < 0.1:
                sample.to_csv('./../data/test/'+file_name+'_'+str(i)+'_'+'.csv', index=False, header=False)
            elif random_float < 0.3 : 
                sample.to_csv('./../data/validation/'+file_name+'_'+str(i)+'_'+'.csv', index=False, header=False)
            else:
                sample.to_csv('./../data/train/'+file_name+'_'+str(i)+'_'+'.csv', index=False, header=False)
            


Chaque eeg = 19 x 31 000
On va les découper en patchs de 1000

In [150]:
pd.read_csv('./../data/kaggle_2/s00.csv', header=None).transpose().to_numpy().shape

(19, 31000)

In [162]:
class Dataset_pretraining(torch.utils.data.Dataset):
    def __init__(self, path_to_data):
        self.path_to_data = path_to_data # par example './../data/kaggle2/'


    def __len__(self):
        return 36 * 30000

    def __getitem__(self, idx): # les eeg font 31000 de long mais ya que 30 000 portions de 1000 de long
        file = idx // 30000
        sample = idx % 30000
        x = pd.read_csv(self.path_to_data + 's' + str(file).zfill(2) + '.csv', header=None).transpose().to_numpy()
        x = x[:, sample: sample+1000]      
        return torch.tensor(x), idx # un extrait de 1000 pts d'un eeg 



In [158]:
class Mydataset(torch.utils.data.Dataset):
    def __init__(self, path_to_data):
        self.path_to_data = path_to_data # par example './../data/train/
        self.X = os.listdir(self.path_to_data) # the list of the files in the train set 


    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        labels=np.array([0,1,1,1,0,1,0,1,1,0,0,1,1,1,0,1,1,1,1,0,1,0,0,1,1,1,1,1,1,1,0,1,1,1,1,1])
        x = pd.read_csv(self.path_to_data + self.X[idx], header=None).to_numpy()
        record_number = int(self.X[idx].split('_')[0])
        return torch.tensor(x), torch.tensor(labels[record_number]) # un eeg sur 1000 échantillonages et le label correspondant



In [229]:
dataloader_train = DataLoader(Mydataset('./../data/train/'), batch_size=5, shuffle=True)
dataloader_val = DataLoader(Mydataset('./../data/validation/'), batch_size=5, shuffle=True)
dataloader_test = DataLoader(Mydataset('./../data/test/'), batch_size=5, shuffle=True)
dataloader_pretraining = DataLoader(Dataset_pretraining('./../data/kaggle_2/'), batch_size=2, shuffle=True)

for batch in dataloader_pretraining :
  print(batch[0].shape)
  print(batch[1])
  break


torch.Size([2, 19, 1000])
tensor([191509, 347033])


In [222]:
import torch
import torch.nn as nn

class EEGFeatureExtractor(nn.Module):
    def __init__(self, feature_dim=100):
        super(EEGFeatureExtractor, self).__init__()
        # input [batch_size, 19, 1000]
        self.conv1= nn.Conv1d(19, 32, 3, padding=1)
        self.conv2= nn.Conv1d(32, 64, 3, padding=1)
        self.conv3= nn.Conv1d(64, 128, 3, padding=1)
        self.conv4= nn.Conv1d(6, 10, 3, padding=1)   
        self.conv5 = nn.Conv1d(64, 15, 3, padding=1)
        self.pool = nn.MaxPool1d(1, 13)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(150, 100)
        self.relu = nn.ReLU()

    def forward(self, x):
        # print('au début', x.shape)
        x = self.relu(self.conv1(x))
        # print(1, x.shape)
        x = self.pool(x)
        # print(2, x.shape)
        x = self.relu(self.conv2(x))
        # print(3, x.shape)
        x = self.pool(x)
        # print(4, x.shape)
        x = torch.transpose(x, 1, 2)
        # print(5, x.shape)
        x = self.relu(self.conv4(x))
        # print(6, x.shape)
        x = torch.transpose(x, 1, 2)
        # print(7, x.shape)
        x = self.conv5(x)
        # print(8, x.shape)
        x = x.flatten(start_dim=1)
        # print(9, x.shape)
        x = self.dropout(x)
        # print('a la fin', x.shape)
        x = self.fc(x)
        return self.relu(x)


In [221]:
for batch in dataloader_train :
  x = batch[0]
  print(x.shape)
  model = EEGFeatureExtractor()
  print(model(x.float()).shape)  
  break

torch.Size([5, 19, 1000])
au début torch.Size([5, 19, 1000])
1 torch.Size([5, 32, 1000])
2 torch.Size([5, 32, 77])
3 torch.Size([5, 64, 77])
4 torch.Size([5, 64, 6])
5 torch.Size([5, 6, 64])
6 torch.Size([5, 10, 64])
7 torch.Size([5, 64, 10])
8 torch.Size([5, 15, 10])
9 torch.Size([5, 150])
a la fin torch.Size([5, 150])
torch.Size([5, 100])


On entraine le features extractor : 


In [ ]:
chemin_vers_sauvegarde = './../models/'

In [241]:
torch.dot(torch.tensor([1, 2]), torch.tensor([1, 2]))

tensor(5)

In [247]:
train_extractor = True
tau = 516 # 1 seconde
model_name='extractor'
if not os.path.exists('./models/'+model_name):
    os.makedirs('./models/'+model_name)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = EEGFeatureExtractor()
n_epochs=200
loss = torch.nn.L1Loss()
param_1 = torch.nn.Parameter(torch.ones(100, requires_grad=True))
param_2 =  torch.nn.Parameter(torch.ones(1, requires_grad=True))
# optimizer = torch.optim.Adam(model.parameters())
optimizer = torch.optim.Adam(
    [{'params': model.parameters()}, {'params': [param_1, param_2]}],
    lr=0.1
)
model.to(device)
loss_train=[]
if train_extractor:
    for epoch in (range(n_epochs)):
        print(epoch)
        losstrain=0
        counttrain=0
        lossval=0
        countval=0
        for batch_x,batch_y in dataloader_train:
            batch_x=batch_x.to(device)
            batch_y = batch_y.long()
            batch_y=batch_y.to(device)
            optimizer.zero_grad()
            first_window = batch_x[0].unsqueeze(0)
            second_window = batch_x[1].unsqueeze(0)
            print(first_window.shape)
            first_prediction = model(first_window.float())
            second_prediction = model(second_window.float())
            label_predicted = torch.dot(param_1, abs(first_prediction - second_prediction).squeeze()) + param_2
           
            if abs(batch_y[0]-batch_y[1])>tau:
                y_pred = torch.tensor([-1]).to(device)
            else:
                y_pred = torch.tensor([1]).to(device) # 1 s'ils sont proches, -1 sinon
            l=torch.log(1+torch.exp(-y_pred*label_predicted))
            counttrain+=1
            print(l.shape)
            l.backward()
            losstrain+=l
            optimizer.step()
        if epoch%10==0:
            print(f'epoch {epoch}, training loss = {losstrain/counttrain}')
        loss_train.append(losstrain/counttrain)
        
    torch.save(model, chemin_vers_sauvegarde+'_final'+'.pth')


    # saving the losses in txt files : 
    loss_list_train = [loss_train[i].detach().cpu().numpy() for i in range(len(loss_train))]



    with open('./losses/loss_train_'+model_name+'.txt', 'w') as f :
        for elt in loss_list_train : 
            f.write(str(elt) + '\n')

0
torch.Size([1, 19, 1000])
torch.Size([1])
torch.Size([1, 19, 1000])
torch.Size([1])


RuntimeError: stack expects each tensor to be equal size, but got [19, 1000] at entry 0 and [19, 750] at entry 4

In [149]:
class EEGClassifier(nn.Module):
    def __init__(self, feature_extractor, num_classes = 2):
        super(EEGClassifier, self).__init__()
        self.feature_extractor = feature_extractor
        self.classifier = nn.Linear(100, num_classes)

    def forward(self, x):
        features = self.feature_extractor(x)
        return self.classifier(features)
